# Comparison of full discretizations using approximate Riemann solvers

How do these solvers impact the solution accuracy when used within a finite volume discretization?  To investigate, we will use them within [PyClaw](http://www.clawpack.org/pyclaw/) to solve two standard test problems for one-dimensional compressible flow:

  1. The Sod shocktube problem.  This is a Riemann problem, which we have considered previously.
  2. The Woodward-Colella blast wave problem.  This initially consists of two Riemann problems, with resulting shock waves of differing strengths.  These shock waves later interact with each other.
  
In this chapter, unlike previous chapters, we include extensive sections of code in the notebook.  This is meant to more easily allow the reader to use these as templates for setting up other problems.  For more information about the software and algorithms used here, see [this paper](https://peerj.com/articles/cs-68/) and references therein.

In [ ]:
%matplotlib inline
import numpy as np
from exact_solvers import Euler
from clawpack import riemann
from utils import riemann_tools
import matplotlib.pyplot as plt
from collections import namedtuple
from ipywidgets import interact
from ipywidgets import widgets
Primitive_State = namedtuple('State', Euler.primitive_variables)
gamma = 1.4
problem_data = {}
problem_data['gamma'] = gamma
problem_data['gamma1'] = gamma - 1.0

In [ ]:
from clawpack.riemann.euler_with_efix_1D_constants import density, momentum, energy, num_eqn

def shocktube(q_l, q_r, N=50, riemann_solver='HLL', solver_type='classic'):

    from clawpack import pyclaw
    from clawpack import riemann

    if riemann_solver == 'Roe':
        rs = riemann.euler_1D_py.euler_roe_1D
    elif riemann_solver == 'HLL':
        rs = riemann.euler_1D_py.euler_hll_1D

    if solver_type == 'classic':
        solver = pyclaw.ClawSolver1D(rs)        
    else:
        solver = pyclaw.SharpClawSolver1D(rs)

    solver.kernel_language = 'Python'
    
    solver.bc_lower[0]=pyclaw.BC.extrap
    solver.bc_upper[0]=pyclaw.BC.extrap

    x = pyclaw.Dimension(-1.0,1.0,N,name='x')
    domain = pyclaw.Domain([x])
    state = pyclaw.State(domain,num_eqn)

    gamma = 1.4
    state.problem_data['gamma']= gamma
    state.problem_data['gamma1']= gamma-1.

    state.problem_data['efix'] = False

    xc = state.grid.p_centers[0]
    
    velocity = (xc<=0)*q_l[1] + (xc>0)*q_r[1]
    pressure = (xc<=0)*q_l[2] + (xc>0)*q_r[2]

    state.q[density ,:] = (xc<=0)*q_l[0] + (xc>0)*q_r[0]
    state.q[momentum,:] = velocity * state.q[density,:]
    state.q[energy  ,:] = pressure/(gamma - 1.) + 0.5 * state.q[density,:] * velocity**2

    claw = pyclaw.Controller()
    claw.tfinal = 0.5
    claw.solution = pyclaw.Solution(state,domain)
    claw.solver = solver
    claw.num_output_times = 10
    claw.keep_copy = True
    claw.verbosity=0

    return claw

In [ ]:
from clawpack.riemann.euler_with_efix_1D_constants import density, momentum, energy, num_eqn

def blastwave(q_l, q_r, N=400, riemann_solver='HLL', solver_type='classic'):

    from clawpack import pyclaw
    from clawpack import riemann

    if riemann_solver == 'Roe':
        rs = riemann.euler_1D_py.euler_roe_1D
    elif riemann_solver == 'HLL':
        rs = riemann.euler_1D_py.euler_hll_1D

    if solver_type == 'classic':
        solver = pyclaw.ClawSolver1D(rs)        
    else:
        solver = pyclaw.SharpClawSolver1D(rs)
        solver.time_integrator = 'SSP33'
        solver.cfl_max = 0.65
        solver.cfl_desired = 0.6
        solver.lim_type = 1
        #solver.char_decomp = 2

    solver.kernel_language = 'Python'
    
    solver.bc_lower[0]=pyclaw.BC.extrap
    solver.bc_upper[0]=pyclaw.BC.extrap

    x = pyclaw.Dimension(0.0,1.0,N,name='x')
    domain = pyclaw.Domain([x])
    state = pyclaw.State(domain,num_eqn)

    gamma = 1.4
    state.problem_data['gamma']= gamma
    state.problem_data['gamma1']= gamma-1.

    state.problem_data['efix'] = False

    xc = state.grid.p_centers[0]

    state.q[density ,:] = 1.
    state.q[momentum,:] = 0.
    state.q[energy  ,:] = ( (xc<0.1)*1.e3 + (0.1<=xc)*(xc<0.9)*1.e-2 + (0.9<=xc)*1.e2 ) / (gamma - 1.)

    claw = pyclaw.Controller()
    claw.tfinal = 0.038
    claw.solution = pyclaw.Solution(state,domain)
    claw.solver = solver
    claw.num_output_times = 10
    claw.keep_copy = True
    claw.verbosity=0

    return claw

## Classic Clawpack algorithm with HLL versus Roe

We compare results obtained with these two solvers within a second-order Lax-Wendroff based scheme with limiters that is the basis of Clawpack.  We start with the Sod shocktube.

In [ ]:
q_l = Euler.conservative_to_primitive(1.,0.,1.)
q_r = Euler.conservative_to_primitive(1./8,0.,1./10)
roe_st = shocktube(q_l,q_r,N=50,riemann_solver='Roe')
roe_st.run()
hll_st = shocktube(q_l,q_r,N=50,riemann_solver='HLL')
hll_st.run()
fine_st = shocktube(q_l,q_r,N=1000,riemann_solver='Roe')
fine_st.run();
xc_st = roe_st.solution.state.grid.p_centers[0]
xc_fine_st = fine_st.solution.state.grid.p_centers[0]

def plot_frame(i):
    fig, ax = plt.subplots(figsize=(12,6))
    ax.set_xlim((-1,1)); ax.set_ylim((0,1.1))
    ax.plot(xc_fine_st,fine_st.frames[i].q[density,:],'-k',lw=1)
    ax.plot(xc_st,hll_st.frames[i].q[density,:],'-ob',lw=2)
    ax.plot(xc_st,roe_st.frames[i].q[density,:],'-or',lw=2)
    plt.legend(['Fine','HLL','Roe'],loc='best')
    plt.show()
    
interact(plot_frame, i=widgets.IntSlider(min=0, max=10, description='Frame'));

The solution labeled "Fine" is computed on a very fine grid and can be considered as the exact solution for purposes of plotting.
As you might expect, the HLL solver smears the middle wave (contact discontinuity) significantly more than the Roe solver does. Perhaps surprisingly, it captures the shock just as accurately as the Roe solver does.

Next we consider the Woodward-Colella blast wave problem.

In [ ]:
q_l = Euler.conservative_to_primitive(1.,0.,1.)
q_r = Euler.conservative_to_primitive(1./8,0.,1./10)
roe_bw = blastwave(q_l,q_r,riemann_solver='Roe')
roe_bw.run()
hll_bw = blastwave(q_l,q_r,riemann_solver='HLL')
hll_bw.run()
fine_bw = blastwave(q_l,q_r,N=2000,riemann_solver='Roe')
fine_bw.run();
xc_bw = roe_bw.solution.state.grid.p_centers[0]
xc_fine_bw = fine_bw.solution.state.grid.p_centers[0]

def plot_frame(i):
    fig, ax = plt.subplots(figsize=(12,6))
    ax.set_xlim((0.,1)); ax.set_ylim((0,20))
    ax.plot(xc_fine_bw,fine_bw.frames[i].q[density,:],'-k',lw=1)
    ax.plot(xc_bw,hll_bw.frames[i].q[density,:],'-ob',lw=2)
    ax.plot(xc_bw,roe_bw.frames[i].q[density,:],'-or',lw=2)
    plt.legend(['Fine','HLL','Roe'],loc='best')
    plt.show()
    
interact(plot_frame, i=widgets.IntSlider(min=0, max=10, description='Frame'));

Again the solutions are fairly similar, though the HLL solution is a bit more smeared.

One should not conclude from these tests that, for instance, the Roe solver is *better* than the HLL solver.  Many factors besides accuracy should be considered, including cost and robustness.  As we have seen the HLL solver is more robust in the presence of near-vacuum states.

## High-order WENO + Runge-Kutta

Next we look at the difference between the HLL and Roe solution when these solvers are employed within a higher-order method of lines discretization using fifth-order WENO and a 4th-order Runge-Kutta scheme.

In [ ]:
q_l = Euler.conservative_to_primitive(1.,0.,1.)
q_r = Euler.conservative_to_primitive(1./8,0.,1./10)
roe2 = shocktube(q_l,q_r,N=50,riemann_solver='Roe',solver_type='sharpclaw')
roe2.run()
hll2 = shocktube(q_l,q_r,N=50,riemann_solver='HLL',solver_type='sharpclaw')
hll2.run()
fine2 = shocktube(q_l,q_r,N=1000,riemann_solver='Roe',solver_type='sharpclaw')
fine2.run();
xc = roe2.solution.state.grid.p_centers[0]
xc_fine2 = fine2.solution.state.grid.p_centers[0]

def plot_frame(i):
    fig, ax = plt.subplots(figsize=(12,6))
    ax.set_xlim((-1,1)); ax.set_ylim((0,1.1))
    ax.plot(xc_fine2,fine2.frames[i].q[density,:],'-k',lw=1)
    ax.plot(xc,hll2.frames[i].q[density,:],'-ob',lw=2)
    ax.plot(xc,roe2.frames[i].q[density,:],'-or',lw=2)
    plt.legend(['Fine','HLL','Roe'],loc='best')
    plt.show()
    
interact(plot_frame, i=widgets.IntSlider(min=0, max=10, description='Frame'));

With higher-order discretizations, the difference in solutions due to using different Riemann solvers is less significant.  This is partly because these high-order schemes use more accurate values as inputs to the Riemann problem, so that in smooth regions the jump between most cells is very small.